# Documentation and Variable Declarations

Purpose: The purpose of this notebook is to map the BRM short names to the BIM and visualize the data with a plotly treemap. 


Author: James Hicks

Date: 11/15/2021

INPUT:
1. The BRM dataset (see brmFileName)

OUTPUT:
1. An HTML file that shows all of the data (see htmlName3)
2. An HTML file that shows ~50% of the data (see htmlName1)
3. An HTML file that shows the other ~50% of the data (see htmlName2)
4. A JSON file that contains all of the data formatted for a d3 treemap

In [1]:
#==========IMPORTS==========
import pandas as pd
import copy
import plotly.express as px
import json
import string

In [62]:
#==========VARIABLE DECLARATIONS==========

#input file names
brmFileName='BRM 7.0.xlsx'

#output file names
#the first 2 html files (part 1 and 2) are created for PNG export for powerpoint slides
#the last html file (all data) is not intended to be converted to a PNG 
htmlName1='BIM Plotly Treemap Part 1.html'
htmlName2='BIM Plotly Treemap Part 2.html'
htmlName3='BIM Plotly Treemap All Data.html'
plotNameBase='VA Business Information Model (BIM)'

jsonFileName='BRM JSON Data.json'

#duplicate name string 
duplicatedName='Research Proposals' #we manually use the full name to fix this duplicate
duplicatedConcatValues=['Write ','Evaluate ']#these are the strings that are concatenated to fix the duplicates

#placeholder value for parents with no children
#this value is also appended to parents with an odd number of children to improve the aesthetic of the final visuals
placeholderValue='\n'
#d3 placeholder is used to sort the output alphabetically. it is replaced with ''
d3Placeholder='zzzzzzzzzz'

#base value for treemap square size and the incremental step value 
#the step value is used to force alphabetical order by changing the treemap value
baseValue=1
stepValue=.0000001

#this number represents the number of characters to use as a cutoff point
#if a parent name reaches the cutoff point, the last word is replaced with the elipseValueStr for a better png output
elipseValue=29
elipseValueStr='...'

#colors for the plotly - these match the original colors
rootColor='#162c52'
parentColor='#5373b0'
childColor='lightgray'

#colors for the d3 visualization 
childColord3="#a6adad"
backgroundColord3="#234b8c"

#the following iFAM numbers have a missing 0 from excel and need to be added
addZeroList=['MEM001','TRT001']

#list of necessary column names for our work
brmColNames=['VA BRM Number','VA BRM Short Name','Level','iFAM Number']

#set height and width for the part 1 and 2 plots to scale well for a powerpoint slide
plotHeight=1600
plotWidth=2850

#plot height and width for the HTML output containing all data
plotHeightAllData=3900
plotWidthAllData=1650

# Read in and Cleaning

In [3]:
#==========READ IN DATA==========
brmRawData=pd.read_excel(brmFileName)

In [4]:
#==========FIX THE MISSING 0'S AND TYPECAST==========

#make a copy to edit
brmData=copy.deepcopy(brmRawData[brmColNames])
#type cast to str
brmData=brmData.astype(str)

#==========BRM ADD MISSING 0'S ========= THIS CAN PROBABLY BE DELETED/COMMENTED OUT IN PALENTIR
#get the rows that are known to have the dropped 0 problem by iFAM number
rowsToChange=copy.deepcopy(brmData[brmData['iFAM Number'].isin(addZeroList)])
#add the 0
rowsToChange['VA BRM Number']=rowsToChange['VA BRM Number']+'0'
#remove the originals
brmData=brmData[~brmData['iFAM Number'].isin(addZeroList)]
#append the fixed rows
brmData=brmData.append(rowsToChange,ignore_index=True)


In [5]:
#==========ISOLATE PARENT AND CHILD DATA==========

#filter to only get levels of interest - these values represent half of the primary key
levelValues=['Capability','Service']#parent, child
brmData=brmData[brmData['Level'].isin(levelValues)]
#declare dataset of parents
capabilityData=brmData[brmData['Level']==levelValues[0]]
#declare dataset of children
serviceData=brmData[brmData['Level']==levelValues[1]]

In [6]:
#==========FIX REPEATED VALUES==========

#find the index of the repeated values to fix
repeatIndexList=list(serviceData.index[serviceData['VA BRM Short Name']==duplicatedName])
#fix the duplicated name by appending the full name to it
serviceData.at[repeatIndexList[0],'VA BRM Short Name']=duplicatedConcatValues[0]+duplicatedName
serviceData.at[repeatIndexList[1],'VA BRM Short Name']=duplicatedConcatValues[1]+duplicatedName

# Map Parents and Children

In [7]:
#==========MAP THE PARENTS TO THE CHILDREN==========

#output dataframe as relationshipDf

data=[]#this is used to hold values and convert into the final dataframe of mapped values
#for each capability, get all of the children and put into the relationshipDf
fullParentList=[]
for i,row in capabilityData.iterrows():#for each capability
    tempBrm=row[0]
    tempParentName=row[1]#get the brm and name
    #now use the BRM # to get the rows in 
    tempKids=serviceData[serviceData['VA BRM Number'].str.startswith(tempBrm+'.')]#add the . to finalize the parent
    #for example, search 1.1.etc will not find 1.10.etc
    #data.append([tempParentName,'']) # if this line gets uncommented it will add in a row for each parent
    #for each row of the kids get generate a list that maps parent to child text
    fullParentList.append(tempParentName)
    for i2,row2 in tempKids.iterrows():#add a row for each child
        tempShort=row2[1]
        tempList=[tempShort,tempParentName]
        data.append(tempList)
    
#combine all of the rows into a dataframe
relationshipDf=pd.DataFrame(data, columns=['Name','Parent'])

# Format for Plotly

In [8]:
#==========ADD THE VALUE COLUMN==========

#add in a value column to make each square the same size in the treemap - this gets modified later
relationshipDf['Value']=baseValue

In [9]:
#==========ADD PLACEHOLDER CHILDREN==========

#for parents with no kids or an odd number, add one placeholder

#create a list of parents without children
noChildList=[]
#for each parent check to see if extra boxes are necessary for a more consistent output
for currentParent in fullParentList:
    tempData=relationshipDf[relationshipDf['Parent']==currentParent]
    tempNKids=len(tempData)
    if(tempNKids%2==1):
        tempData={'Name':placeholderValue,'Parent':currentParent,'Value':1}
        relationshipDf=relationshipDf.append(tempData,ignore_index=True)
    elif(tempNKids==0):
        noChildList.append(currentParent)
        tempData={'Name':placeholderValue,'Parent':currentParent,'Value':1}
        relationshipDf=relationshipDf.append(tempData,ignore_index=True)

In [10]:
#==========FORCE ALPHABETICAL ORDER USING THE VALUE COLUMN==========

#this becomes a list containing each parent dataset 
smallData=[]

#for each mini dataset
for currentParent in fullParentList:
    tempData=copy.deepcopy(relationshipDf[relationshipDf['Parent']==currentParent])
    tempNKids=len(tempData)
    
    #this case shouldn't occur
    if(tempNKids==0):
        print('There is a parent with no children that should have a placeholder value:',currentParent)
        next
    elif(tempNKids==1):#one child does not need any size adjustments
        smallData.append(tempData)
    else:
        #sort by name value and remove the old index column
        tempData=tempData.sort_values('Name')
        tempData=tempData.reset_index()
        del tempData['index']
        #get the list of values
        tempValues=list(tempData['Value'])
        tempNames=list(tempData['Name'])
        firstName=tempNames[0]
        for i in range(0,len(tempValues)):
            if(i==0):#base
                if(firstName==placeholderValue):
                    tempValues[i]=baseValue-stepValue*len(tempValues)
            else:#make each one smaller as the loop progresses
                tempValues[i]=baseValue-stepValue*i
        #set the new values
        tempData['Value']=tempValues
        #add the rows to the list of new rows
        smallData.append(tempData)
        
#combine all of the new rows to overwrite the old data
relationshipDf=pd.concat(smallData,ignore_index=True)

In [11]:
#==========ADD HTML DISPLAY VALUES FOR THE PLOTLY==========

#add html new lines for cleaner output
relationshipDf['Name']=relationshipDf['Name'].str.replace(' ','<br>')#replace spaces with html breakline
relationshipDf['Name']=relationshipDf['Name'].str.replace('/','<br>')

#replace multiple breaklines in a row with one 
relationshipDf['Name']=relationshipDf['Name'].str.replace('<br><br>','<br>')#replace double new line with one
relationshipDf['Name']=relationshipDf['Name'].str.replace('<br><br>','<br>')
#replace any html tags that are alone with the placeholder value 
relationshipDf['Name']=relationshipDf['Name'].str.replace('<b><br></b>',placeholderValue)

#add bold tag to make it look/feel the same as the old vis
relationshipDf['Name']='<b>'+relationshipDf['Name'].astype(str)+'</b>'
relationshipDf['Parent']='<b>'+relationshipDf['Parent'].astype(str)+'</b>'
relationshipDf['Name']=relationshipDf['Name'].str.replace('<b>'+placeholderValue+'</b>',placeholderValue)


In [12]:
#==========CREATE COLOR MAPPING==========

#for each child set the color
fullChildList=list(relationshipDf['Name'])#+['<b>'+placeholderValue+'</b>']
colorDict={key: childColor for key in fullChildList}

#anything else (parents) gets the parent color 
colorDict['(?)']=parentColor #this represents the parent nodes
colorDict[placeholderValue]=parentColor

In [13]:
#==========SORT THE DATAFRAME ALPHABETICALLY==========

#sort to find the midpoint without removing children from their parents

#sort by parent value 
relationshipDf=relationshipDf.sort_values('Parent')
#reset index
relationshipDf=relationshipDf.reset_index()
del relationshipDf['index']

In [14]:
#==========FIND THE MIDPOINT FOR PART 1 AND 2==========

#create a range to look through parents with
totalLen=len(relationshipDf)
lowerBound=int(totalLen/2)-3
upperBound=int(totalLen/2)+10
#get the values in range
shortParentList=list(relationshipDf[lowerBound:upperBound]['Parent'])
#set an index and last value to compare
midIndex=0
lastValue=shortParentList[0]
for value in shortParentList:
    if(value!=lastValue):
        break
    else:
        midIndex=midIndex+1
        
#final output midpoint is midIndex
midIndex=midIndex+lowerBound

In [15]:
#==========FIND ELIPSE VARIABLE NAMES==========

#find all of the parent names that are too long based on the elipse cutoff variable 
# and add to the nochild list to shorten them

longNameList=[]
for value in fullParentList:
    tempLen=len(value)
    if(tempLen>elipseValue):
        noChildList.append(value)

In [16]:
#==========REPLACE ELIPSE STRING VALUES==========

#for each name with no children or a name that is too long, drop the last word and add "..."

noChildListShort=[]
for i in range(0,len(noChildList)):
    tempValue=noChildList[i]
    tempValue=tempValue.rsplit(' ',1)[0]
    tempValue=tempValue+elipseValueStr
    noChildListShort.append(tempValue)
    
#then create a dictionary for easy replacement in the dataframe
replaceParentDict={}
for key in noChildList:
    for value in noChildListShort:
        replaceParentDict[key]=value
        noChildListShort.remove(value)
        break

In [17]:
#==========CREATE A NEW COLUMN WITH THE SHORTENED NAMES==========

#add a column with slightly different parent names for the powerpoint version

#duplicate to make changes
relationshipDf['ParentShort']=relationshipDf['Parent']
#remove html bold
relationshipDf['ParentShort']=relationshipDf['ParentShort'].str.replace('<b>','')
relationshipDf['ParentShort']=relationshipDf['ParentShort'].str.replace('</b>','')
#replace the last word of all parents without children
relationshipDf=relationshipDf.replace({'ParentShort' : replaceParentDict})
#add the bold back 
relationshipDf['ParentShort']='<b>'+relationshipDf['ParentShort'].astype(str)+'</b>'

# Create the Visualizations

## HTML All Data

In [18]:
#==========CREATE TREEMAP USING ALL DATA==========

fig=px.treemap(relationshipDf,path=['Parent','Name'],values='Value',
               color='Name',color_discrete_map=colorDict)
fig.update_traces(root_color=rootColor)
fig.update_layout(uniformtext=dict(minsize=13, mode='show'))
fig.update_layout(autosize=False,width=plotWidthAllData,height=plotHeightAllData)
fig.update_layout(title_text=plotNameBase,font=dict(size=13,color="Black"))
fig.data[0].hovertemplate='Name=%{label}<br>Parent=%{parent}'
fig.write_html(htmlName3)

## HTML Parts 1 & 2

In [19]:
#==========CREATE TREEMAP PART 1 USING 50% OF THE DATA ==========

#create a second treemap using the first ~50% of the data
fig=px.treemap(relationshipDf[0:midIndex],path=['ParentShort','Name'],values='Value',
               color='Name',color_discrete_map=colorDict)
fig.update_traces(root_color=rootColor)
fig.update_layout(uniformtext=dict(minsize=13, mode='show'))
fig.update_layout(autosize=False,width=plotWidth,height=plotHeight)
fig.update_layout(title_text=plotNameBase+' Part 1/2',font=dict(size=13,color="Black"))
fig.data[0].hovertemplate='Name=%{label}<br>Parent=%{parent}'
fig.write_html(htmlName1)

In [20]:
#==========CREATE TREEMAP PART 2 USING 50% OF THE DATA ==========

#create a second treemap using the other ~50% of the data
fig=px.treemap(relationshipDf[midIndex:len(relationshipDf)],path=['ParentShort','Name'],values='Value',
               color='Name',color_discrete_map=colorDict)
fig.update_traces(root_color=rootColor)
fig.update_layout(uniformtext=dict(minsize=13, mode='show'))
fig.update_layout(autosize=False,width=plotWidth,height=plotHeight)
fig.update_layout(title_text=plotNameBase+' Part 2/2',font=dict(size=13,color="Black"))
fig.data[0].hovertemplate='Name=%{label}<br>Parent=%{parent}'
fig.write_html(htmlName2)

# Export Data for D3

In [72]:
#==========DECLARE FUNCTIONS TO CREATE THE JSON OUTPUT==========

def find_element(children_list,name):
    """
    Find element in children list
    if exists or return none
    """
    for i in children_list:
        if i["name"] == name:
            return i
    #If not found return None
    return None

def add_node(path,value,nest,colNameVal,groupVal,colorVal):
    """
    The path is a list.  Each element is a name that corresponds 
    to a level in the final nested dictionary.  
    """
    #Get first name from path
    this_name = path.pop(0)
    #Does the element exist already?
    element = find_element(nest["children"], this_name)

    #If the element exists, we can use it, otherwise we need to create a new one
    if element:
        if len(path)>0:
            add_node(path,value, element,colNameVal,groupVal,colorVal)
    #Else it does not exist so create it and return its children
    else:
        if len(path) == 0:
            nest["children"].append({"name": this_name, "value": value,"colname":colNameVal,"group":groupVal,"color":colorVal})
        else:
            #Add new element
            nest["children"].append({"name": this_name, "children":[]})
            #Get added element 
            element = nest["children"][-1]
            #Still elements of path left so recurse
            add_node(path,value, element,colNameVal,groupVal,colorVal)

In [54]:
#==========MODIFY THE PLOTLY DATA FOR D3 FORMATTING - INITIAL FORMATTING==========

#Make a copy to edit
d3Data=copy.deepcopy(relationshipDf)

#in each string column, remove the HTML values that were put in for the plotly formatting
colList=list(d3Data.columns)
colList.remove('Value')
for key in colList:
    d3Data[key]=d3Data[key].str.replace('<b>','')
    d3Data[key]=d3Data[key].str.replace('</b>','')
    d3Data[key]=d3Data[key].str.replace('<br>',' ')
    #replace the d3 placeholers with zzzs to sort
    d3Data[key]=d3Data[key].str.replace('\n',d3Placeholder)
#drop the short column
d3Data=d3Data.drop('ParentShort',1)
#add in the colname column to match the d3 expected format
d3Data['colname']='level3'

In [55]:
#==========MODIFY THE PLOTLY DATA FOR D3 FORMATTING - ADD THE GROUP COLUMN==========

#add in the group column
groupCol=list(d3Data['Parent'])

#set a current parent value for comparison
currentParent=groupCol[0]
#create a list of 52 characters to act as the group labels. In the future if this needs to be expanded, make the alphabet list longer
alphabet=list(string.ascii_uppercase)+list(string.ascii_lowercase)
alphaIndex=0

for i in range(0,len(groupCol)):
    currentValue=groupCol[i]
    #if the current value matches the parent, set the value
    if(currentValue==currentParent):
        groupCol[i]=alphabet[alphaIndex]
    #the value doesn't match, so increment and set new current
    else:
        currentParent=currentValue
        alphaIndex=alphaIndex+1
        #if(i==len(groupCol)):
        groupCol[i]=alphabet[alphaIndex]
d3Data['group']=groupCol

In [56]:
#==========MODIFY THE PLOTLY DATA FOR D3 FORMATTING - SORT ALPHABETICALLY==========

#sort the d3 dataframe alphabetically by 2 cols
d3Data=d3Data.sort_values(by = ['Parent', 'Name'], ascending = [True, True], na_position = 'first')
#replace the placeholders with empty string now that the sort is finished
d3Data['Name']=d3Data['Name'].str.replace(d3Placeholder,'')

In [64]:
#==========MODIFY THE PLOTLY DATA FOR D3 FORMATTING - ADD COLOR COLUMN==========

#for each row where the name is placeholder, set the color to be the same as the background
currentPlaceholder=''
#loop through all of the names and compare the current name to the placeholder value
nameList=list(d3Data['Name'])
for i in range(0,len(nameList)):
    currentValue=nameList[i]
    if(currentValue==currentPlaceholder):#set as background
        nameList[i]=backgroundColord3
    else:
        nameList[i]=childColord3#set as normal child node color
d3Data['Color']=nameList

In [73]:
#==========CREATE THE JSON OUTPUT==========
output = {"children": []}

levels = ["Parent","Name"]
for row in d3Data.iterrows():
    r = row[1]
    path = list(r[levels])
    value = r["Value"]
    colnameValue=r['colname']
    groupValue=r['group']
    colorValue=r['Color']
    add_node(path,value,output,colnameValue,groupValue,colorValue)

#print(json.dumps(d, sort_keys=False,indent=2))

In [74]:
#==========SAVE THE JSON OUTPUT==========
with open(jsonFileName,'w') as outputFile:
    json.dump(output,outputFile,indent=4)